# FO

In [1]:
# ToDo: Remove pathlib, only here to find correct paths.
# import pathlib
# print(pathlib.Path().absolute())

# As we are writing in a seperate folder we need to add paths to the other modules not contained in the current system paths.
import sys
sys.path.insert(1,'/home/jetbot')
sys.path.insert(1,'/home/jetbot/monodepth2')

In [2]:
from jetson import utils, inference
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from PIL import Image as pil
import cv2

from jetbot import Camera, bgr8_to_jpeg, Robot

import time
import numpy as np

jetson.utils.__init__.py
jetson.inference.__init__.py


In [5]:
camwidth, camheight = 224, 224
cam = Camera.instance(width=camwidth, height=camheight)

RuntimeError: Could not initialize camera.  Please see error trace.

## Setup

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
if __name__ == '__main__':
    print(device)

model = models.resnet50(pretrained=True)
model = model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])
normalize = transforms.Normalize(mean, stdev)

def preprocess(camera_value): # Preprocessing for camera format --> Network input format
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB) #(x, i)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

'''transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])'''

model.eval()

with open('classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]
    
camwidth, camheight = 224, 224
cam = Camera.instance(width=camwidth, height=camheight)

rob = Robot()

In [ ]:
with open('classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]

## YEET

In [ ]:
def update(change):
    # global robot, prob_blocked
    try:
        x = change['new'] 
        x = preprocess(x)
        out = model(x)

        # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
        _, index = torch.max(out, 1)
        if index in [967, 968, 504, 505]:
            rob.forward(0.4)
            time.sleep(0.5)
            rob.stop()
        pct = F.softmax(out, dim=1)[0] * 100
        print(classes[index[0]], round(pct[index[0]].item(), 3))
        time.sleep(0.1)
    except KeyboardInterrupt:
        cam.stop()
        print('Loop successfully ended.')

In [ ]:
update({'new': cam.value})
cam.observe(update, names='value')

## Return Max

In [ ]:
k, index = torch.max(out, 1)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print(classes[index[0]], percentage[index[0]].item())

### Top 5

In [ ]:
_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print([(classes[idx], percentage[idx].item()) for idx in indices[0][:5]])

# Pseudokode:

Timeline:
- Ta inn en gjenstand
- Kjør rundt
- Se etter samme gjenstand
- Identifiser som samme gjenstand
- Kjør bort til den


Kode:

Ta inn gjenstand, gjør estimering, lagre ID

som del av update<cam.value> spesial funskjonen: konstant identifiser gjenstander.

Hvis den gjetter samme id som er lagret, ovveride(break) og gå til oppsøkingsfunksjon.



In [ ]:
while True:
    try:
        img = pil.fromarray(cam.value.astype('uint8'),'RGB') #.rotate(180)
        # img = Image.open("a.jpg")
        img_t = transform(img)
        batch_t = torch.unsqueeze(img_t, 0).cuda()
        out = model(batch_t)
        
        _, index = torch.max(out, 1)
        percentage = F.softmax(out, dim=1)[0] * 100
        print(classes[index[0]], percentage[index[0]].item())
        time.sleep(0.1)
    except KeyboardInterrupt:
        cam.stop()
        print('Loop successfully ended.')
        break

## Auto-forsøk

In [ ]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])
normalize = transforms.Normalize(mean, stdev)

def preprocess(camera_value): # Preprocessing for camera format --> Network input format
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB) #(x, i)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = transform(x)
    x = x.to(device)
    x = x[None, ...]
    return x

transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])


In [ ]:
def prep(cam_val):
    global device, normalize
    x = pil.fromarray(cam_val.astype('uint8'),'RGB')
    x.show()
    x = transform(x)
    x = torch.unsqueeze(img_t, 0)
    x = x.to(device)
    x = x[None, ...]
    return x



def update(change):
    # global robot, prob_blocked
    x = change['new'] 
    print(x.shape)
    x = prep(x)
    print(x.shape)
    out = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    _, index = torch.max(out, 1)
    pct = F.softmax(out, dim=1)[0] * 100
    print(classes[index[0]], round(pct[index[0]].item(), 3))
    time.sleep(0.01)

In [ ]:
def update(change):
    # global robot, prob_blocked
    x = change['new'] 
    x = preprocess(x)
    out = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    _, index = torch.max(out, 1)
    pct = F.softmax(out, dim=1)[0] * 100
    print(classes[index[0]], round(pct[index[0]].item(), 3))
        
    time.sleep(0.01)

# update({'new': cam.value})
# cam.observe(update, names='value')

In [ ]:
cam.unobserve(update, names='value')
